#Dataset preparation

In [ ]:
#DAVIS Dataset
import kagglehub




path = kagglehub.dataset_download("mengzj/davis-data")

print("Path to dataset files:", path)

In [ ]:
import cv2
import os
import numpy as np
from tqdm import tqdm

# Paths
DATASET_PATH = "/root/.cache/kagglehub/datasets/mengzj/davis-data/versions/1/DAVIS/JPEGImages/480p"
OUTPUT_COLOR_PATH = "dataset/color/"
OUTPUT_BW_PATH = "dataset/bw/"
IMG_SIZE = (128, 128)

def extract_frames():
    os.makedirs(OUTPUT_COLOR_PATH, exist_ok=True)
    os.makedirs(OUTPUT_BW_PATH, exist_ok=True)

    video_folders = sorted(os.listdir(DATASET_PATH))

    for video in tqdm(video_folders, desc="Processing Videos"):
        video_path = os.path.join(DATASET_PATH, video)
        frame_files = sorted(os.listdir(video_path))
        frames=0

        for i, frame_name in enumerate(frame_files):
            frame_path = os.path.join(video_path, frame_name)
            frame = cv2.imread(frame_path)

            if frame is None:
                continue

            frame = cv2.resize(frame, IMG_SIZE)

            # Save color frame
            color_save_path = os.path.join(OUTPUT_COLOR_PATH, f"{video}_{i}.png")
            cv2.imwrite(color_save_path, frame)

            # Convert to grayscale and save
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            bw_save_path = os.path.join(OUTPUT_BW_PATH, f"{video}_{i}.png")
            cv2.imwrite(bw_save_path, gray_frame)

            frames += 1
            if frames==100:
              break

    print(f"Dataset prepared {len(os.listdir(OUTPUT_COLOR_PATH))} frames extracted.")

if __name__ == "__main__":
    extract_frames()




```
# This is formatted as code
```

#Model Colorization

#Model Training

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
#from colorization_model import build_colorization_model
from tqdm import tqdm

# Paths
BW_PATH = "dataset/bw/"
COLOR_PATH = "dataset/color/"
MODEL_PATH = "colorization_300.keras"
IMG_SIZE = (128, 128)

# Load dataset
def load_dataset():
    bw_images, color_images = [], []

    file_names = sorted(os.listdir(BW_PATH))

    for file in tqdm(file_names, desc="Loading Frames"):
        bw_img = cv2.imread(os.path.join(BW_PATH, file), cv2.IMREAD_GRAYSCALE)
        color_img = cv2.imread(os.path.join(COLOR_PATH, file))

        if bw_img is None or color_img is None:
            continue

        bw_img = cv2.resize(bw_img, IMG_SIZE) / 255.0
        color_img = cv2.resize(color_img, IMG_SIZE) / 255.0

        bw_images.append(img_to_array(bw_img).reshape(128, 128, 1))  # Gray input
        color_images.append(img_to_array(color_img))  # RGB output

    return np.array(bw_images), np.array(color_images)

# Train model
#def train_and_save_model(epochs=10, batch_size=32):
X_train, Y_train = load_dataset()
print(f"Loaded {len(X_train)} frames for training.\nStarting Training...")
model = build_colorization_model()
history = model.fit(X_train, Y_train, epochs=100, batch_size=64, validation_split=0.2)

model.save(MODEL_PATH)
print(f"Model saved at {MODEL_PATH}")

#if __name__ == "__main__":
  #  train_and_save_model(epochs=50,batch_size=32)

#Video Colorization

# New Section

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.losses import MeanSquaredError

def build_colorization_model(input_shape=(128, 128, 1)):
    model = models.Sequential([
        # Encoder
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.AveragePooling2D((2, 2)),

        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.AveragePooling2D((2, 2)),

        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.AveragePooling2D((2, 2)),

        # Bottleneck
        layers.Conv2D(512, (3, 3), activation='relu', padding='same'),

        # Decoder
        layers.Conv2DTranspose(256, (3, 3), strides=2, activation='relu', padding='same'),
        layers.Conv2DTranspose(128, (3, 3), strides=2, activation='relu', padding='same'),
        layers.Conv2DTranspose(64, (3, 3), strides=2, activation='relu', padding='same'),

        layers.Conv2D(3, (3, 3), activation='tanh', padding='same')  # Output: 3 color channels
    ])

    optimizer = Nadam(learning_rate=0.0001)

    model.compile(optimizer=optimizer, loss=MeanSquaredError(), metrics=["accuracy"])
    return model

In [ ]:
import cv2
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.losses import MeanSquaredError

# Paths
MODEL_PATH = "colorization_300.keras"
INPUT_VIDEO = "bw_4.mp4"
OUTPUT_VIDEO = "colorized_video.mp4"

IMG_SIZE = (512, 512)  # Corrected to match training image size (128x128)

# Load the model with the correct loss function
custom_objects = {"mse": MeanSquaredError()}
model = tf.keras.models.load_model(MODEL_PATH, custom_objects=custom_objects)

def process_video():
    cap = cv2.VideoCapture(INPUT_VIDEO)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, fps, (width, height))

    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Step 1: Convert input frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Step 2: Resize to model input size
        gray_frame_resized = cv2.resize(gray_frame, IMG_SIZE) / 255.0
        gray_frame_resized = img_to_array(gray_frame_resized).reshape(1, 512, 512, 1)

        # Step 3: Predict color frame
        color_frame = model.predict(gray_frame_resized)[0]
        color_frame = (color_frame * 255).astype(np.uint8)  # Bring back to [0,255]

        # Step 4: Resize predicted frame back to original frame size
        color_frame = cv2.resize(color_frame, (width, height))

        # Step 5: Save frame
        out.write(color_frame)
        frame_count += 1

        print(f"Processed frame {frame_count}")

    cap.release()
    out.release()
    print(f"Video colorization completed: {OUTPUT_VIDEO}")

if __name__ == "__main__":
    process_video()


# Deployment (For Images)

In [ ]:
!pip install gradio --quiet

import gradio as gr
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
import cv2
import tensorflow as tf

# Load trained model

model = tf.keras.models.load_model("colorization_300.keras", custom_objects={"mse": tf.keras.losses.MeanSquaredError()})

IMG_SIZE = (128, 128)

def colorize_image(input_img):
# Convert image to grayscale
gray = cv2.cvtColor(input_img, cv2.COLOR_RGB2GRAY)
gray = cv2.resize(gray, IMG_SIZE)
gray_input = gray.astype("float32") / 255.0
gray_input = img_to_array(gray_input).reshape(1, 128, 128, 1)

```
# Predict color image
output = model.predict(gray_input)[0]
output = np.clip(output * 255, 0, 255).astype("uint8")
output_resized = cv2.resize(output, (input_img.shape[1], input_img.shape[0]))
return output_resized

```

# Launch Gradio UI

gr.Interface(
fn=colorize_image,
inputs=gr.Image(type="numpy", label="Upload Grayscale Image"),
outputs=gr.Image(type="numpy", label="Colorized Image"),
title="Black & White to Color Image Colorization"
).launch()